In [1]:
import tensorflow as tf


# Get list of available GPU devices
gpu_devices = tf.config.list_physical_devices('GPU')

if gpu_devices:
    print("GPU is available")
    # Additional information about GPUs
    for gpu in gpu_devices:
        print(gpu)
else:
    print("GPU is not available")

GPU is available
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [2]:

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv("C:/Users/RAZER/Downloads/dataset/aptos/train.csv")
test = pd.read_csv("C:/Users/RAZER/Downloads/dataset/aptos/test.csv")
#submission= pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard


N_CLASSES = train['diagnosis'].nunique()
N_CLASSES




# Preprocecss data
train["id_code"] = train["id_code"].apply(lambda x: x + ".png")
test["id_code"] = test["id_code"].apply(lambda x: x + ".png")
train['diagnosis'] = train['diagnosis'].astype('str')
train.head()





train_datagen=ImageDataGenerator(rescale=1./255, 
                                validation_split=0.2,
                                horizontal_flip=True)


train_generator=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="C:/Users/RAZER/Downloads/dataset/aptos/train_images/",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=16,
    class_mode="categorical",
    target_size=(224, 224),
    subset='training')





valid_generator=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="C:/Users/RAZER/Downloads/dataset/aptos/train_images/",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=16,
    class_mode="categorical",    
    target_size=(224, 224),
    subset='validation')




test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(  
        dataframe=test,
        directory = "C:/Users/RAZER/Downloads/dataset/aptos/test_images/",
        x_col="id_code",
        target_size=(224, 224),
        batch_size=16,
        shuffle=False,
        class_mode=None)

Found 2930 validated image filenames belonging to 5 classes.
Found 732 validated image filenames belonging to 5 classes.
Found 1928 validated image filenames.


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50


import tensorflow as tf
from tensorflow.keras import layers, models

In [13]:
from keras.models import load_model
aptos_resnet_model = load_model("C:/Users/RAZER/Documents/Python Scripts/saved_models/retina/aptos_resnet.h5")

In [18]:
"""loss, accuracy = aptos_resnet_model.evaluate(train_generator)
print("Train loss:", loss, "Train accuracy:", accuracy)

test_loss, test_accuracy = aptos_resnet_model.evaluate(test_generator)
print("Test loss:", test_loss, "Test accuracy:", test_accuracy)

val_loss, val_accuracy = aptos_resnet_model.evaluate(valid_generator)
print("Validation loss:", val_loss, "Val accuracy:", val_accuracy)"""

184/184 [==============================] - 373s 2s/step - loss: 0.8132 - accuracy: 0.7249
Train loss: 0.8131960034370422 Train accuracy: 0.7249146699905396
121/121 [==============================] - 97s 791ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Test loss: 0.0 Test accuracy: 0.0
46/46 [==============================] - 108s 2s/step - loss: 0.8278 - accuracy: 0.7117
Validation loss: 0.8277633190155029 Val accuracy: 0.7117486596107483


In [29]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import shap
import numpy as np



# Add custom classification layers on top of ResNet50
N_CLASSES = 5  # Define the number of classes for your problem



# Assuming valid_generator is your validation data generator and properly defined before this code
# Prepare the dataset for SHAP analysis
num_samples = 100  # Adjust the number of samples based on your computation resources
X_shap, _ = next(valid_generator)  # Extract a subset of data
X_shap = X_shap[:num_samples]

# Using SHAP's DeepExplainer to handle high-dimensional data like images
explainer = shap.DeepExplainer(aptos_resnet_model, X_shap)
shap_values = explainer.shap_values(X_shap)

# Visualize SHAP values
# Note: For high-dimensional data, the visualization part might need customization
shap.summary_plot(shap_values, X_shap.reshape(X_shap.shape[0], -1), plot_type='bar')



keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


StagingError: in user code:

    File "c:\Users\RAZER\anaconda3\envs\Viper\lib\site-packages\shap\explainers\_deep\deep_tf.py", line 259, in grad_graph  *
        x_grad = tape.gradient(out, shap_rAnD)

    LookupError: gradient registry has no entry for: shap_AddV2
